## Dataset Automotores

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
#Levanto tabla de ramos

ramos_df=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_AM\tabla_ramos.xlsx', index_col = 'cod_subramo')

In [3]:
ramos_df.head(2)

,cod_ramo,des_ramo,des_subramo
cod_subramo,,,
1.010.99,1.010.00,Incendio,Incendio - Incendio
1.020.01,1.020.00,Combinado Familiar e Integral,Comb. Fam. e Int. - Comb. Fam.


ANEXO 12 (Todos los ramos)

In [4]:
# Anexo 12 Primas - el primer trimestre tiene que ser 3

anexo12_df=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_AM\ANEXO_12.xlsx')

anexo12_df.cod_cia=anexo12_df.cod_cia.astype('O')

anexo12_df=anexo12_df.replace(to_replace=np.nan, value=0)

anexo12_df=anexo12_df.drop(anexo12_df[anexo12_df['cod_subramo']=='3.000.99'].index)
anexo12_df=anexo12_df.drop(anexo12_df[anexo12_df['cod_subramo']=='9999'].index)
anexo12_df=anexo12_df.drop(anexo12_df[anexo12_df['cod_subramo']==0].index)

In [5]:
anexo12_df.head(1)

,cod_cia,des_cia,año,cod_subramo,Primas_Emitidas,Primas_Cedidas_Reaseguro,Compromisos_Tecnicos_del_Ej_Total,Compromisos_Tecnicos_del_Ej_AC_Reaseg,Compromisos_Tecnicos_del_Ej_AC_Asegurad,Compromisos_Tecnicos_del_Ej_Ant_Total,Compromisos_Tecnicos_del_Ej_Ant_AC_Reaseg,Compromisos_Tecnicos_del_Ej_Ant_AC_Asegurad,Primas_Devengadas_SD,Primas_Reaseguros_Activos,Primas_Cedidas_a_Retrocesion,Compromisos_Tecnicos_del_Ej,Compromisos_Tecnicos_del_Ej_Ant,Primas_Devengadas_RA,Primas_Devengadas_Total
0,2,SMG COMPAÑIA ARGENTINA DE SEGUROS S.A.,2008-3,1.010.99,6024283,5691406,5988471,4633711,1354760,4090236,3047768,1042468,20585,0,0,0,0,0,20585


In [6]:
# Crea la columna Ramo y la completa buscando la información de la tabla de ramos

anexo12_ramo=anexo12_df

anexo12_ramo['ramo']=[ramos_df.loc[anexo12_df.cod_subramo[i],'des_ramo'] for i in anexo12_df.index]

In [7]:
# Crea una tabla del tipo "tabla dinámica de excel"
# Agrupa filas: por ID y ramo, columnas: año (año=aaaa-t) y suma las primas emitidas

anexo12_pivot=anexo12_ramo.pivot_table(index=['cod_cia','ramo'], values=['Primas_Emitidas'], columns=['año'], aggfunc=np.sum)

In [8]:
# Reemplaza los missing por nulos

anexo12_pivot=anexo12_pivot.replace(to_replace=np.nan, value=0)

In [9]:
#Creación base trimestral desacumulada

# Creo el esqueleto donde después va a incluir la información con un bucle. Lo que le estoy pidiendo es que utilice como esqueleto
# la misma estructura que tengo hasta ahora en 'anexo12_pivot' y que le modifique el nombre de las columnas a partir de 'multi_index'


multi_index=pd.MultiIndex(levels=[list(anexo12_pivot.columns.levels[0]), list(anexo12_pivot.columns.levels[1])],
                                    labels=list(anexo12_pivot.columns.labels),
                                    names=['variable', 'año'])


anexo12_trim=pd.DataFrame(index=anexo12_pivot.index , columns=multi_index)

In [10]:
# Defino la función que voy a utilizar para hacer el cálculo

# La función indica que si el período de análidis es 3, entonces que retorne el mismo monto de prima emitida. 
# Sino que al valor del período actual le reste el período anterior.

def trim(row,col,año):
    if anexo12_trim.loc[anexo12_trim.iloc[row].name[0]][col][año].name.split('-')[1]=='3':
        
        return anexo12_pivot.loc[anexo12_trim.iloc[row].name[0],anexo12_trim.iloc[row].name[1]][col][año]
    
    else:
        
        return anexo12_pivot.loc[anexo12_trim.iloc[row].name[0],anexo12_trim.iloc[row].name[1]][col][año]-anexo12_pivot.loc[anexo12_trim.iloc[row].name[0],anexo12_trim.iloc[row].name[1]][col][anexo12_trim.columns.levels[1][list(anexo12_trim.columns.levels[1]).index(año)-1]]
 

In [11]:
# Defino el bucle for al que voy a aplicar la función

variables=['Primas_Emitidas']

for j in range(len(anexo12_trim.index)):
    for col in variables:
        for año in list(anexo12_trim.columns.levels[1]):
            anexo12_trim.iloc[j][col][año]=trim(j,col,año)

In [12]:
# Creo dos variables
anexo12_trim['cod_cia']=[anexo12_trim.iloc[i].name[0] for i in range(len(anexo12_trim.index))]
anexo12_trim['ramo']=[anexo12_trim.iloc[i].name[1] for i in range(len(anexo12_trim.index))]

In [13]:
anexo12_trim_melt_totales=anexo12_trim.melt(id_vars=['cod_cia','ramo'])

anexo12_trim_melt_totales=anexo12_trim_melt_totales.drop({'variable','ramo'}, 1)

anexo12_trim_melt_totales=anexo12_trim_melt_totales.rename(columns = {'value':'Primas_Emitidas_Desagregadas_Totales'})

anexo12_trim_melt_totales=anexo12_trim_melt_totales.groupby(['cod_cia','año'], as_index=False)["Primas_Emitidas_Desagregadas_Totales"].sum()

ANEXO 12 (Auutomotor)

In [14]:
# Anexo 12 Primas - el primer trimestre tiene que ser 3

anexo12_df=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\AUTOMOTOR\ANEXO_12.xlsx')

anexo12_df.cod_cia=anexo12_df.cod_cia.astype('O')

anexo12_df=anexo12_df[(anexo12_df.cod_subramo=='1.030.01')|(anexo12_df.cod_subramo=='1.030.02')|(anexo12_df.cod_subramo=='1.030.03')]

In [15]:
# Crea la columna Ramo y la completa buscando la información de la tabla de ramos

anexo12_ramo=anexo12_df

anexo12_ramo['ramo']=[ramos_df.loc[anexo12_df.cod_subramo[i],'des_ramo'] for i in anexo12_df.index]

In [16]:
# Crea una tabla del tipo "tabla dinámica de excel"
# Agrupa filas: por ID y ramo, columnas: año (año=aaaa-t) y suma las primas emitidas

anexo12_pivot=anexo12_ramo.pivot_table(index=['cod_cia','ramo'], values=['Primas_Emitidas'], columns=['año'], aggfunc=np.sum)

In [17]:
# Reemplaza los missing por nulos

anexo12_pivot=anexo12_pivot.replace(to_replace=np.nan, value=0)

In [18]:
#Creación base trimestral desacumulada

# Creo el esqueleto donde después va a incluir la información con un bucle. Lo que le estoy pidiendo es que utilice como esqueleto
# la misma estructura que tengo hasta ahora en 'anexo12_pivot' y que le modifique el nombre de las columnas a partir de 'multi_index'


multi_index=pd.MultiIndex(levels=[list(anexo12_pivot.columns.levels[0]), list(anexo12_pivot.columns.levels[1])],
                                    labels=list(anexo12_pivot.columns.labels),
                                    names=['variable', 'año'])


anexo12_trim=pd.DataFrame(index=anexo12_pivot.index , columns=multi_index)


In [19]:
# Defino la función que voy a utilizar para hacer el cálculo

# La función indica que si el período de análidis es 3, entonces que retorne el mismo monto de prima emitida. 
# Sino que al valor del período actual le reste el período anterior.

def trim(row,col,año):
    if anexo12_trim.loc[anexo12_trim.iloc[row].name[0]][col][año].name.split('-')[1]=='3':
        
        return anexo12_pivot.loc[anexo12_trim.iloc[row].name[0],anexo12_trim.iloc[row].name[1]][col][año]
    
    else:
        
        return anexo12_pivot.loc[anexo12_trim.iloc[row].name[0],anexo12_trim.iloc[row].name[1]][col][año]-anexo12_pivot.loc[anexo12_trim.iloc[row].name[0],anexo12_trim.iloc[row].name[1]][col][anexo12_trim.columns.levels[1][list(anexo12_trim.columns.levels[1]).index(año)-1]]

In [20]:
# Defino el bucle for al que voy a aplicar la función

variables=['Primas_Emitidas']

for j in range(len(anexo12_trim.index)):
    for col in variables:
        for año in list(anexo12_trim.columns.levels[1]):
            anexo12_trim.iloc[j][col][año]=trim(j,col,año)

In [21]:
anexo12_trim['cod_cia']=[anexo12_trim.iloc[i].name[0] for i in range(len(anexo12_trim.index))]
anexo12_trim['ramo']=[anexo12_trim.iloc[i].name[1] for i in range(len(anexo12_trim.index))]

In [22]:
# Obtengo el dataset con Ramo, Año y Prima Emitida Desagragada

anexo12_trim_melt_autom=anexo12_trim.melt(id_vars=['cod_cia','ramo'])

anexo12_trim_melt_autom=anexo12_trim_melt_autom.drop({'variable','ramo'}, 1)

anexo12_trim_melt_autom=anexo12_trim_melt_autom.rename(columns = {'value':'Primas_Emitidas_Desagregadas'})

In [23]:
# Agrupo el Anexo 12 por año y compañía

anexo12_df_agrup=anexo12_df.groupby(['cod_cia','año'], as_index=False)["Primas_Emitidas","Primas_Devengadas_Total"].sum()

In [24]:
# Junto datasets: Prima emitida desagregada con Prima Emitida

Dataset_AUTOM=anexo12_trim_melt_autom.merge(anexo12_trim_melt_totales, how='inner', on=['cod_cia','año'])

Dataset_AUTOM=Dataset_AUTOM.merge(anexo12_df_agrup, how='inner', on=['cod_cia','año'])


ANEXO 13

In [25]:
anexo13_df=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\AUTOMOTOR\ANEXO_13.xlsx', usecols= "A:D,H,V")
anexo13_df.cod_cia=anexo13_df.cod_cia.astype('O')

anexo13_df['Siniestros_pagados netos']=anexo13_df['Pagados_Netos']
anexo13_df=anexo13_df.drop('Pagados_Netos',1)
anexo13_df['Siniestros_devengados']=anexo13_df['Total_Devengada_Ej_RA']
anexo13_df=anexo13_df.drop('Total_Devengada_Ej_RA',1)

anexo13_df=anexo13_df[(anexo13_df.cod_subramo=='1.030.01')|(anexo13_df.cod_subramo=='1.030.02')|(anexo13_df.cod_subramo=='1.030.03')]
anexo13_df_agrup=anexo13_df.groupby(['cod_cia','año'], as_index=False)['Siniestros_pagados netos','Siniestros_devengados'].sum()

In [26]:
# Junto datasets: Anexo 12 + Anexo 13

Dataset_AUTOM=Dataset_AUTOM.merge(anexo13_df_agrup, how='outer', on=['cod_cia','año'])

ANEXO 14

In [27]:
anexo14_df=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\AUTOMOTOR\ANEXO_14.xlsx', usecols='A:E,G,N')
anexo14_df.cod_cia=anexo14_df.cod_cia.astype('O')

anexo14_df=anexo14_df[(anexo14_df.cod_subramo=='1.030.01')|(anexo14_df.cod_subramo=='1.030.02')|(anexo14_df.cod_subramo=='1.030.03')]
anexo14_df_agrup=anexo14_df.groupby(['cod_cia','año'], as_index=False)['Sub_Total_GP','Sub_total_GE'].sum()

In [28]:
# Junto datasets: Anexo 12 + Anexo 13 + Anexo 14

Dataset_AUTOM=Dataset_AUTOM.merge(anexo14_df_agrup, how='outer', on=['cod_cia','año'])

ANEXO 3

In [29]:
anexo3_df=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\AUTOMOTOR\ANEXO_3.xlsx', usecols='A:E,G')
anexo3_df.cod_cia=anexo3_df.cod_cia.astype('O')

anexo3_df=anexo3_df[(anexo3_df.cod_subramo=='1.030.01')|(anexo3_df.cod_subramo=='1.030.02')|(anexo3_df.cod_subramo=='1.030.03')]
anexo3_df_agrup=anexo3_df.groupby(['cod_cia','año'], as_index=False)['Premios_a_Cobrar','Prevision_para_Incobrabilidad'].sum()

In [30]:
anexo3_df.head()

,cod_cia,des_cia,año,cod_subramo,Premios_a_Cobrar,Prevision_para_Incobrabilidad
1514,2,SMG COMPAÑIA ARGENTINA DE SEGUROS S.A.,2014-1,1.030.01,189712844,-2400093
1515,25,ROYAL & SUN ALLIANCE SEGUROS (ARGENTINA) S.A.,2014-1,1.030.01,948424912,-15141711
1516,32,BOSTON COMPAÑÍA ARGENTINA DE SEGUROS SOCIEDAD ...,2014-1,1.030.01,85574341,-2474163
1517,36,ALLIANZ ARGENTINA COMPAÑIA DE SEGUROS S.A.,2014-1,1.030.01,316521017,-7298176
1518,39,QBE SEGUROS LA BUENOS AIRES SOCIEDAD ANONIMA,2014-1,1.030.01,454123277,-9783150


In [31]:
# Junto datasets: Anexo 12 + Anexo 13 + Anexo 14 + Anexo 3

Dataset_AUTOM=Dataset_AUTOM.merge(anexo3_df_agrup, how='outer', on=['cod_cia','año'])

ANEXO 8

In [32]:
anexo8_df=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\AUTOMOTOR\ANEXO_8.xlsx', usecols='A:H')
anexo8_df.cod_cia=anexo8_df.cod_cia.astype('O')

anexo8_df=anexo8_df[(anexo8_df.cod_subramo=='1.030.01')|(anexo8_df.cod_subramo=='1.030.02')|(anexo8_df.cod_subramo=='1.030.03')]
anexo8_df_agrup=anexo8_df.groupby(['cod_cia','año'], as_index=False)['RSP_Administrativos','RSP_Juicios','RSP_Mediaciones', 'IBNR'].sum()

In [33]:
# Junto datasets: Anexo 12 + Anexo 13 + Anexo 14 + Anexo 3 + Anexo 8

Dataset_AUTOM=Dataset_AUTOM.merge(anexo8_df_agrup, how='outer', on=['cod_cia','año'])

ANEXO 11

In [34]:
anexo11_df=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\AUTOMOTOR\ANEXO_11.xlsx', usecols='A:D,H')
anexo11_df.cod_cia=anexo11_df.cod_cia.astype('O')

anexo11_df=anexo11_df[(anexo11_df.cod_subramo=='1.030.01')|(anexo11_df.cod_subramo=='1.030.02')|(anexo11_df.cod_subramo=='1.030.03')]
anexo11_df_agrup=anexo11_df.groupby(['cod_cia','año'], as_index=False)['Reserva_Tecnica_por_Insuficiencia_de_Primas'].sum()

In [35]:
# Junto datasets: Anexo 12 + Anexo 13 + Anexo 14 + Anexo 3 + Anexo 8 + Anexo 11

Dataset_AUTOM=Dataset_AUTOM.merge(anexo11_df_agrup, how='outer', on=['cod_cia','año'])

Cantidad de Vehículos Expuestos

In [36]:
ve_exp_df=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\AUTOMOTOR\Cantidad de Vehículos Expuestos.xlsx')

In [37]:
# Junto datasets: Anexo 12 + Anexo 13 + Anexo 14 + Anexo 3 + Anexo 8 + Anexo 11 + Cantidad de Vehículos Expuestos 

Dataset_AUTOM=Dataset_AUTOM.merge(ve_exp_df, how='outer', on=['cod_cia','año'])

Cantidad de Juicios por Ramo

In [38]:
cant_jui_df=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\output\Cantidad de juicios automotor.xlsx')

In [39]:
cant_jui_df.head()

,cod_cia,año,Juicios
0,2,2017-4,1024
1,2,2018-1,1079
2,2,2018-2,1191
3,2,2018-3,1281
4,2,2018-4,1385


In [40]:
# Junto datasets: Anexo 12 + Anexo 13 + Anexo 14 + Anexo 3 + Anexo 8 + Anexo 11 + Cantidad de Vehículos Expuestos + Cantidad de Juicios

Dataset_AUTOM=Dataset_AUTOM.merge(cant_jui_df, how='left', on=['cod_cia','año'])

Compañías

In [41]:
# Importo nombre compañías

Dataset_compañías=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\AUTOMOTOR\desc_cia.xlsx', sheet_name='AM')

In [42]:
Dataset_AUTOM=Dataset_AUTOM.merge(Dataset_compañías, how='left', on='cod_cia')

In [43]:
# Filtro compañías activas

comp_act=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\AUTOMOTOR\cias_activas.xlsx')

In [44]:
Dataset_AUTOM=Dataset_AUTOM.merge(comp_act, how='inner', on='cod_cia')

In [45]:
cias=[6,202,227,381,404,574,812,820,821,829,855,857,858,744]

Dataset_AUTOM['cias_activas']=[1 if np.isin(Dataset_AUTOM.cod_cia.iloc[i],cias)==True else 0 for i in range(len(Dataset_AUTOM.index))]

Dataset_AUTOM=Dataset_AUTOM[(Dataset_AUTOM.cias_activas==0)]


Filtros y Preparación final

In [46]:
Dataset_AUTOM['year'] = pd.DatetimeIndex(Dataset_AUTOM['año']).year

In [47]:
Dataset_AUTOM= Dataset_AUTOM.drop(Dataset_AUTOM[Dataset_AUTOM.year < 2017].index)

In [48]:
Dataset_AUTOM= Dataset_AUTOM.drop(Dataset_AUTOM[Dataset_AUTOM['año']=='2017-1'].index)
Dataset_AUTOM= Dataset_AUTOM.drop(Dataset_AUTOM[Dataset_AUTOM['año']=='2017-2'].index)
Dataset_AUTOM= Dataset_AUTOM.drop(Dataset_AUTOM[Dataset_AUTOM['año']=='2017-3'].index)

In [49]:
cols = list(Dataset_AUTOM.columns.values)
cols

['cod_cia',
 'año',
 'Primas_Emitidas_Desagregadas',
 'Primas_Emitidas_Desagregadas_Totales',
 'Primas_Emitidas',
 'Primas_Devengadas_Total',
 'Siniestros_pagados netos',
 'Siniestros_devengados',
 'Sub_Total_GP',
 'Sub_total_GE',
 'Premios_a_Cobrar',
 'Prevision_para_Incobrabilidad',
 'RSP_Administrativos',
 'RSP_Juicios',
 'RSP_Mediaciones',
 'IBNR',
 'Reserva_Tecnica_por_Insuficiencia_de_Primas',
 'cantidad_vehiculos',
 'Juicios',
 'des_cia',
 'denominacion_corta',
 'cias_activas',
 'year']

In [50]:
# Reordeno las columnas
reord= ['cod_cia', 
 'denominacion_corta',
 'des_cia',
 'año',
 'Primas_Emitidas_Desagregadas',
 'Primas_Emitidas_Desagregadas_Totales',
 'Primas_Emitidas',
 'Primas_Devengadas_Total',
 'Siniestros_pagados netos',
 'Siniestros_devengados',
 'Sub_Total_GP',
 'Sub_total_GE',
 'Premios_a_Cobrar',
 'Prevision_para_Incobrabilidad',
 'RSP_Administrativos',
 'RSP_Juicios',
 'RSP_Mediaciones',
 'IBNR',
 'Reserva_Tecnica_por_Insuficiencia_de_Primas',
 'cantidad_vehiculos',
 'Juicios']

Dataset_AUTOM=Dataset_AUTOM[reord]

In [51]:
# Reemplaza los missing por nulos

Dataset_AUTOM=Dataset_AUTOM.replace(to_replace=np.nan, value=0)

In [52]:
Dataset_AUTOM.to_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\output\Dataset_AUTOM.xlsx', sheet_name='Dataset_AUTOM', index=False)